In [ ]:
KEYWORD_SEARCH_API = "https://pm-copilot-macro-gcl-osdp-gsd-gsd-delivery-staging.aibi-americas-002.dyn.nesc.nokia.net/api/v1/search/keywords/detect"

KPI_SEARCH_API = "https://pm-copilot-macro-gcl-osdp-gsd-gsd-delivery-staging.aibi-americas-002.dyn.nesc.nokia.net/api/v1/semantic/search"

QB_SEARCH_API = "https://pm-copilot-macro-gcl-osdp-gsd-gsd-delivery-staging.aibi-americas-002.dyn.nesc.nokia.net/api/v1/semantic/search"

SCHEMA_WORLD_VIEW = "https://pm-copilot-macro-gcl-osdp-gsd-gsd-delivery-staging.aibi-americas-002.dyn.nesc.nokia.net/api/v1/dev/schema-world-view"

In [ ]:
user_query = [
"For each Region and General Contractor, calculate the total number of active New Site Build projects, the On-Air Completion Rate (%), the average Site Completion Cycle Time from Construction Start to On-Air, and the count of sites where Civil Start was delayed by more than 60 days after Entitlement Completion.",
"List all sites scheduled to start RAN construction within the next 15 days that do not have a Customer Purchase Order (CPO) and do not have a Supplier Purchase Order (SPO), including Region, Market, Project ID, General Contractor, Site Class, Hard Cost approval status, and Building Permit approval status.",
"By Region and Market, compute the Call Test Pass Rate and Swap First Time Right (FTR) percentage for all swapped sites, along with the total number of swapped sites, the number of sites that failed call tests, and the number of sites with more than one HSE supervision entry."
"For each General Contractor and Region, determine the total number of Civil Starts, the number of Civil Completion delays exceeding 21 days, the average Civil construction duration, and the number of sites where Civil construction has not started within 60 days after Entitlement Completion for Site Mod and New Build projects."
"Identify all sites where Construction is complete but Integration has not started, or where Integration has started but not completed, or where the delay between Integration completion and On-Air exceeds 3 days, and return Region, Market, Project ID, General Contractor, Site Class, and Crane or Lift requirement status."
]

In [ ]:
import requests

# API endpoint
url = KEYWORD_SEARCH_API

# Query parameter
params = {
    "sentence": user_query[0]
}

# Optional headers (add Authorization if required)
headers = {
    "Accept": "application/json",
    # "Authorization": "Bearer YOUR_ACCESS_TOKEN"  # Uncomment if needed
}

try:
    response = requests.get(url, params=params, headers=headers, timeout=30)

    # Raise exception for HTTP errors
    response.raise_for_status()

    data = response.json()

    print("Input Sentence:", data.get("input_sentence"))
    print("Detected Count:", data.get("detected_count"))
    print("Detected Keywords:")
    
    keyword_names = []
    for keyword in data.get("detected_keywords", []):
        print(f" - {keyword['keyword_name']} (Matched: {keyword.get('matched_terms')})")
        if keyword['keyword_name']:
            keyword_names.append(keyword['keyword_name'])

except requests.exceptions.RequestException as e:
    print("API Request Failed:", str(e))


Input Sentence: For each Region and General Contractor, calculate the total number of active New Site Build projects, the On-Air Completion Rate (%), the average Site Completion Cycle Time from Construction Start to On-Air, and the count of sites where Civil Start was delayed by more than 60 days after Entitlement Completion.
Detected Count: 3
Detected Keywords:
 - New Site Build (Matched: ['New Site Build'])
 - Site (Matched: ['Site'])
 - EC (Matched: ['EC'])


In [6]:
import requests

# API endpoint
url = KPI_SEARCH_API
payload = {
    "query": user_query[0],   # or replace with a string
    "table": "kpi",
    "top_k": 8
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    # "Authorization": "Bearer YOUR_ACCESS_TOKEN"
}

try:
    response = requests.post(url, json=payload, headers=headers, timeout=30)
    response.raise_for_status()

    data = response.json()

    print("Query:", data.get("query"))
    print("Total Results:", data.get("total_results"))

    # List to store KPI names
    kpi_names = []

    for kpi in data.get("results", []):
        content = kpi.get("content", {})
        kpi_name = content.get("kpi_name")

        if kpi_name:
            kpi_names.append(kpi_name)

    print("\nExtracted KPI Names:")
    print(kpi_names)

except requests.exceptions.RequestException as e:
    print("API Request Failed:", str(e))

Query: For each Region and General Contractor, calculate the total number of active New Site Build projects, the On-Air Completion Rate (%), the average Site Completion Cycle Time from Construction Start to On-Air, and the count of sites where Civil Start was delayed by more than 60 days after Entitlement Completion.
Total Results: 8

Extracted KPI Names:
['Site Completion Cycle Time', 'Civil Start Delay (>60 Days)', 'Civil Completion Delay', 'On-Air Pending after Construction done', 'On-Air Completion Rate (%)', 'Entitlement Completion Rate', '% of sites meets Cx complete SLA - NSD (MB)', 'Civil NTP Acceptance Cycle Time']


In [12]:
import requests

# API endpoint
url = KPI_SEARCH_API
payload = {
    "query": user_query[0],   # or replace with a string
    "table": "question_bank",
    "top_k": 3
}

headers = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    # "Authorization": "Bearer YOUR_ACCESS_TOKEN"
}

try:
    response = requests.post(url, json=payload, headers=headers, timeout=30)
    response.raise_for_status()

    data = response.json()

    print("Query:", data.get("query"))
    print("Total Results:", data.get("total_results"))

    # List to store KPI names
    qb_names = []

    for qb in data.get("results", []):
        content = qb.get("content", {})
        qb_name = content.get("question")

        if qb_name:
             qb_names.append(qb_name)


except requests.exceptions.RequestException as e:
    print("API Request Failed:", str(e))

Query: For each Region and General Contractor, calculate the total number of active New Site Build projects, the On-Air Completion Rate (%), the average Site Completion Cycle Time from Construction Start to On-Air, and the count of sites where Civil Start was delayed by more than 60 days after Entitlement Completion.
Total Results: 3


In [11]:
content

{'question': 'List all sites ready for Civil start in next 8 weeks',
 'question_id': '941553',
 'created_at': '2026-02-17 12:55:33.708253',
 'sql': 'select rgn_region AS "Region",m_market AS "Market",pj_project_id AS "Project_Id",\npj_a_4200_pull_construction_po_number_finish::date AS "PO Available", \npj_p_4250_civil_construction_start_finish::date AS "Civil Start Forecast Date"\nfrom public.ndpd_mbt_tmobile_macro_combined \nwhere pj_a_4200_pull_construction_po_number_finish IS NOT NULL AND  \npj_project_type = \'N-NSD Macro\'AND \npj_a_4250_civil_construction_start_finish IS NULL\nAND pj_p_4250_civil_construction_start_finish::date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL \'8 weeks\';',
 'updated_at': '2026-02-17 12:55:33.708257'}

In [13]:
import requests
from requests.auth import HTTPBasicAuth

# API endpoint
url = SCHEMA_WORLD_VIEW

# Request payload
payload = {
    "kpi_names": kpi_names,
    "question_texts": qb_names,
    "keyword_names": keyword_names
}

# Headers
headers = {
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Basic Authentication
auth = HTTPBasicAuth("Any_string", "pwc_macro_team")

try:
    response = requests.post(
        url,
        json=payload,
        headers=headers,
        auth=auth,
        timeout=30
    )

    response.raise_for_status()
    data = response.json()

    print("Success:", data.get("success"))
    print("\nWorld View:\n")
    print(data.get("world_view"))

except requests.exceptions.RequestException as e:
    print("API Request Failed:", str(e))


Success: True

World View:

{'schema_name': 'pwc_macro_staging_schema', 'input': {'kpis': ['Site Completion Cycle Time', 'Civil Start Delay (>60 Days)', 'Civil Completion Delay', 'On-Air Pending after Construction done', 'On-Air Completion Rate (%)', 'Entitlement Completion Rate', '% of sites meets Cx complete SLA - NSD (MB)', 'Civil NTP Acceptance Cycle Time'], 'questions': ['List the CxS to On Air high cycle time sites region wise', 'List the sites in WIP status and no. of days since WIP', 'List all sites ready for Civil start in next 8 weeks'], 'keywords': ['New Site Build', 'Site', 'EC']}, 'kpi_contexts': [{'kpi_name': 'Site Completion Cycle Time', 'description': 'Measures the average number of days from project start (Cx Start) to On-Air for Site Mods and New Builds', 'formula': 'Average(PJ:Completion Objective Actual Date" – PJ(A 4225) Construction Start-Finish)', 'logic': 'Consider base table as mbt_tracker and match with pm_tracker then In MBT ‘Project Type’ column should be co

In [14]:
data

{'success': True,
 'world_view': {'schema_name': 'pwc_macro_staging_schema',
  'input': {'kpis': ['Site Completion Cycle Time',
    'Civil Start Delay (>60 Days)',
    'Civil Completion Delay',
    'On-Air Pending after Construction done',
    'On-Air Completion Rate (%)',
    'Entitlement Completion Rate',
    '% of sites meets Cx complete SLA - NSD (MB)',
    'Civil NTP Acceptance Cycle Time'],
   'questions': ['List the CxS to On Air high cycle time sites region wise',
    'List the sites in WIP status and no. of days since WIP',
    'List all sites ready for Civil start in next 8 weeks'],
   'keywords': ['New Site Build', 'Site', 'EC']},
  'kpi_contexts': [{'kpi_name': 'Site Completion Cycle Time',
    'description': 'Measures the average number of days from project start (Cx Start) to On-Air for Site Mods and New Builds',
    'formula': 'Average(PJ:Completion Objective Actual Date" – PJ(A 4225) Construction Start-Finish)',
    'logic': 'Consider base table as mbt_tracker and match

In [15]:
def build_text2sql_context(world_view: dict) -> dict:
    """
    Distills the raw world_view JSON into a compact context
    optimized for a text-to-SQL LLM. Drops all_columns, validation
    blocks, related_tables, and other metadata noise.
    """
    wv = world_view.get("world_view", world_view)

    context = {
        "schema_name": wv.get("schema_name"),
        "user_input": wv.get("input"),
    }

    # --- KPI contexts: keep only what the LLM needs to write SQL ---
    context["kpis"] = []
    for kpi in wv.get("kpi_contexts", []):
        context["kpis"].append({
            "kpi_name": kpi["kpi_name"],
            "description": kpi["description"],
            "formula": kpi["formula"],
            "logic": kpi["logic"],
            "tables": kpi["tables_involved"],
            "columns": kpi["matched_columns_by_table"],
        })

    # --- Question bank: keep reference SQL as few-shot examples ---
    context["reference_questions"] = []
    for q in wv.get("question_contexts", []):
        entry = {
            "question": q["question"],
            "tables": q["tables_involved"],
            "columns": q["matched_columns_by_table"],
        }
        if q.get("reference_sql"):
            entry["reference_sql"] = q["reference_sql"]
        context["reference_questions"].append(entry)

    # --- Keywords: domain vocabulary for the LLM ---
    context["keywords"] = []
    for kw in wv.get("keyword_contexts", []):
        context["keywords"].append({
            "keyword": kw["keyword_name"],
            "description": kw["description"],
            "synonyms": kw.get("synonyms"),
            "columns": kw["matched_columns_by_table"],
        })

    # --- Focused tables: only referenced columns, no all_columns ---
    context["tables"] = {}
    for table_name, info in wv.get("focused_tables", {}).items():
        context["tables"][table_name] = info["referenced_columns"]

    return context

In [16]:
from toon import encode

print(encode(build_text2sql_context(data)))

schema_name: pwc_macro_staging_schema
user_input:
  kpis[8]: Site Completion Cycle Time,Civil Start Delay (>60 Days),Civil Completion Delay,On-Air Pending after Construction done,On-Air Completion Rate (%),Entitlement Completion Rate,% of sites meets Cx complete SLA - NSD (MB),Civil NTP Acceptance Cycle Time
  questions[3]: List the CxS to On Air high cycle time sites region wise,List the sites in WIP status and no. of days since WIP,List all sites ready for Civil start in next 8 weeks
  keywords[3]: New Site Build,Site,EC
kpis[8]:
  - kpi_name: Site Completion Cycle Time
    description: Measures the average number of days from project start (Cx Start) to On-Air for Site Mods and New Builds
    formula: "Average(PJ:Completion Objective Actual Date\" – PJ(A 4225) Construction Start-Finish)"
    logic: "Consider base table as mbt_tracker and match with pm_tracker then In MBT ‘Project Type’ column should be considered as either Site Mod or New Build. The ‘PJ(A 4225) Construction Start-Fi

In [ ]:
"""
Text-to-SQL LangGraph Agent (Fully Async)
==========================================
Flow:
  1. Parallel API calls → keyword detection, KPI search, question bank search
  2. Schema world-view API  → full context from backend
  3. Distill world-view     → drop noise via build_text2sql_context
  4. Generate SQL            → GPT-5-mini with high reasoning
  5. Execute SQL             → run against PostgreSQL via asyncpg
  6. On error → retry with error context (max 3 attempts)
  7. Return DataFrame + per-node timestamps
"""

import asyncio
import json
import time
from datetime import datetime, timezone
from typing import Any, TypedDict

import asyncpg
import httpx
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import END, StateGraph

from world_view_utils import build_text2sql_context

# ──────────────────────────────────────────────────────────────────────
# Configuration
# ──────────────────────────────────────────────────────────────────────

DB_CONFIG = {
    "host": "10.208.241.129",
    "port": 30924,
    "user": "osdp_pwc_read_role",
    "password": "Nokia@osdp@pwc@2025",
    "database": "postgres",
    "server_settings": {"search_path": "pwc_macro_staging_schema,public"},
}

BASE_URL = (
    "https://pm-copilot-macro-gcl-osdp-gsd-gsd-delivery-staging"
    ".aibi-americas-002.dyn.nesc.nokia.net"
)
KEYWORD_SEARCH_API = f"{BASE_URL}/api/v1/search/keywords/detect"
KPI_SEARCH_API = f"{BASE_URL}/api/v1/semantic/search"
QB_SEARCH_API = f"{BASE_URL}/api/v1/semantic/search"
SCHEMA_WORLD_VIEW_API = f"{BASE_URL}/api/v1/dev/schema-world-view"

WORLD_VIEW_AUTH = ("Any_string", "pwc_macro_team")

MAX_SQL_RETRIES = 3

# LLM
llm = ChatOpenAI(
    api_key="NONE",
    model="gpt-5-mini",
    base_url="https://llmgateway-qa-api.nokia.com/v1.2/",
    default_headers={
        "api-key": (
            "eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9."
            "eyJ1c2VyTmFtZSI6IlJvbGxvdXRBZ2VudERldlRvb2wiLCJPYmplY3RJRCI6IjNGRUY4NzI0LUE5RTItNDMyQy1COTk4LTU0NDMwMEZBQjcxMyIsIndvcmtTcGFjZU5hbWUiOiJWUjE4NTdSb2xsb3V0QWdlbnREZXZTcGFjZSIsIm5iZiI6MTc3MDg4NTYyNCwiZXhwIjoxODAyNDIxNjI0LCJpYXQiOjE3NzA4ODU2MjR9."
            "HvO_kB6UNwcNSV4B_MXuh9OK54rYtRRDMPH5Y1EuOhc"
        ),
        "workspacename": "VR1857RolloutAgentDevSpace",
    },
    temperature=0,
    verbose=True,
    reasoning_effort="high",
)


# ──────────────────────────────────────────────────────────────────────
# State
# ──────────────────────────────────────────────────────────────────────

class AgentState(TypedDict):
    user_query: str
    # parallel search results
    keyword_names: list[str]
    kpi_names: list[str]
    qb_names: list[str]
    # world view
    raw_world_view: dict
    context: dict            # distilled
    # sql generation / execution
    generated_sql: str
    sql_error: str
    retry_count: int
    # final output
    result_df: Any           # pandas DataFrame or None
    final_status: str
    # per-node timing
    node_timestamps: dict    # {node_name: {start: str, end: str, duration_s: float}}


# ──────────────────────────────────────────────────────────────────────
# Timestamp helper
# ──────────────────────────────────────────────────────────────────────

def _now_iso() -> str:
    return datetime.now(timezone.utc).isoformat(timespec="milliseconds")


def _merge_timestamps(existing: dict, node_name: str, start: str, end: str, duration: float) -> dict:
    """Return a new dict with the node entry added (avoids mutating state)."""
    updated = {**existing}
    updated[node_name] = {"start": start, "end": end, "duration_s": round(duration, 3)}
    return updated


# ──────────────────────────────────────────────────────────────────────
# Node helpers – async API calls
# ──────────────────────────────────────────────────────────────────────

async def _fetch_keywords(client: httpx.AsyncClient, query: str) -> list[str]:
    resp = await client.get(
        KEYWORD_SEARCH_API,
        params={"sentence": query},
        headers={"Accept": "application/json"},
        timeout=30,
    )
    resp.raise_for_status()
    data = resp.json()
    return [
        kw["keyword_name"]
        for kw in data.get("detected_keywords", [])
        if kw.get("keyword_name")
    ]


async def _fetch_kpis(client: httpx.AsyncClient, query: str) -> list[str]:
    resp = await client.post(
        KPI_SEARCH_API,
        json={"query": query, "table": "kpi", "top_k": 8},
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=30,
    )
    resp.raise_for_status()
    data = resp.json()
    return [
        r["content"]["kpi_name"]
        for r in data.get("results", [])
        if r.get("content", {}).get("kpi_name")
    ]


async def _fetch_question_bank(client: httpx.AsyncClient, query: str) -> list[str]:
    resp = await client.post(
        QB_SEARCH_API,
        json={"query": query, "table": "question_bank", "top_k": 3},
        headers={"Accept": "application/json", "Content-Type": "application/json"},
        timeout=30,
    )
    resp.raise_for_status()
    data = resp.json()
    return [
        r["content"]["question"]
        for r in data.get("results", [])
        if r.get("content", {}).get("question")
    ]


# ──────────────────────────────────────────────────────────────────────
# Graph Nodes (all async with timestamps)
# ──────────────────────────────────────────────────────────────────────

async def parallel_search(state: AgentState) -> dict:
    """Fire keyword, KPI, and question-bank searches in parallel."""
    t_start, iso_start = time.perf_counter(), _now_iso()

    query = state["user_query"]
    async with httpx.AsyncClient() as client:
        keyword_names, kpi_names, qb_names = await asyncio.gather(
            _fetch_keywords(client, query),
            _fetch_kpis(client, query),
            _fetch_question_bank(client, query),
        )

    duration = time.perf_counter() - t_start
    return {
        "keyword_names": keyword_names,
        "kpi_names": kpi_names,
        "qb_names": qb_names,
        "node_timestamps": _merge_timestamps(
            state.get("node_timestamps", {}), "parallel_search", iso_start, _now_iso(), duration
        ),
    }


async def fetch_world_view(state: AgentState) -> dict:
    """Call schema-world-view endpoint with collected search results."""
    t_start, iso_start = time.perf_counter(), _now_iso()

    payload = {
        "kpi_names": state["kpi_names"],
        "question_texts": state["qb_names"],
        "keyword_names": state["keyword_names"],
    }
    async with httpx.AsyncClient() as client:
        resp = await client.post(
            SCHEMA_WORLD_VIEW_API,
            json=payload,
            headers={
                "Accept": "application/json",
                "Content-Type": "application/json",
            },
            auth=WORLD_VIEW_AUTH,
            timeout=30,
        )
        resp.raise_for_status()
        data = resp.json()

    duration = time.perf_counter() - t_start
    return {
        "raw_world_view": data,
        "node_timestamps": _merge_timestamps(
            state.get("node_timestamps", {}), "fetch_world_view", iso_start, _now_iso(), duration
        ),
    }


async def distill_context(state: AgentState) -> dict:
    """Apply build_text2sql_context to strip noise from the world view."""
    t_start, iso_start = time.perf_counter(), _now_iso()

    context = build_text2sql_context(state["raw_world_view"])

    duration = time.perf_counter() - t_start
    return {
        "context": context,
        "node_timestamps": _merge_timestamps(
            state.get("node_timestamps", {}), "distill_context", iso_start, _now_iso(), duration
        ),
    }


# ───────── SQL Generation Prompt ─────────

SQL_SYSTEM_PROMPT = """\
You are an expert PostgreSQL query generator for a telecom project management database.

## Rules
- Output ONLY a single valid PostgreSQL SELECT statement. No markdown, no explanation.
- Use the schema: {schema_name}
- Prefix all table names with the schema, e.g. {schema_name}.table_name
- Use only the tables and columns listed in the context below.
- Cast date columns explicitly with ::date when comparing dates.
- Use CURRENT_DATE for "today".
- Use double-quoted aliases for readability, e.g. AS "Region".
- NEVER use SELECT *; always list specific columns.
- For KPI calculations, follow the formula and logic provided exactly.
- Reference SQL examples are provided as few-shot guides — adapt them, don't copy blindly.
- Have a limt for 200 Rows Maximum

## Context
{context_json}
"""

SQL_RETRY_PROMPT = """\
The previous SQL query failed with this database error:

```
{error}
```

Previous failing query:
```sql
{sql}
```

Fix the query based on the error. Output ONLY the corrected PostgreSQL SELECT statement.
Use the same context and rules as before.
"""


async def generate_sql(state: AgentState) -> dict:
    """Generate SQL from the distilled context (or fix a previous error)."""
    t_start, iso_start = time.perf_counter(), _now_iso()

    ctx = state["context"]
    schema = ctx.get("schema_name", "pwc_macro_staging_schema")
    context_json = json.dumps(ctx, indent=2, default=str)

    messages = [
        SystemMessage(content=SQL_SYSTEM_PROMPT.format(
            schema_name=schema,
            context_json=context_json,
        )),
    ]

    if state.get("sql_error") and state.get("generated_sql"):
        messages.append(HumanMessage(content=SQL_RETRY_PROMPT.format(
            error=state["sql_error"],
            sql=state["generated_sql"],
        )))
    else:
        messages.append(HumanMessage(content=(
            f"Write a PostgreSQL query to answer this question:\n\n"
            f"{state['user_query']}"
        )))

    response = await llm.ainvoke(messages)
    sql = response.content.strip()

    # Strip markdown fences if the LLM wraps them
    if sql.startswith("```"):
        sql = "\n".join(sql.split("\n")[1:])
    if sql.endswith("```"):
        sql = "\n".join(sql.split("\n")[:-1])
    sql = sql.strip()

    duration = time.perf_counter() - t_start
    retry_count = state.get("retry_count", 0)
    node_label = f"generate_sql" if retry_count == 0 else f"generate_sql_retry_{retry_count}"
    return {
        "generated_sql": sql,
        "sql_error": "",
        "node_timestamps": _merge_timestamps(
            state.get("node_timestamps", {}), node_label, iso_start, _now_iso(), duration
        ),
    }


async def execute_sql(state: AgentState) -> dict:
    """Run the generated SQL against PostgreSQL via asyncpg, return a DataFrame."""
    t_start, iso_start = time.perf_counter(), _now_iso()
    sql = state["generated_sql"]

    try:
        conn = await asyncpg.connect(**DB_CONFIG)
        try:
            rows = await conn.fetch(sql)
            if rows:
                columns = list(rows[0].keys())
                df = pd.DataFrame([dict(r) for r in rows], columns=columns)
            else:
                df = pd.DataFrame()
        finally:
            await conn.close()

        duration = time.perf_counter() - t_start
        retry_count = state.get("retry_count", 0)
        node_label = "execute_sql" if retry_count == 0 else f"execute_sql_retry_{retry_count}"
        return {
            "result_df": df,
            "sql_error": "",
            "final_status": "success",
            "node_timestamps": _merge_timestamps(
                state.get("node_timestamps", {}), node_label, iso_start, _now_iso(), duration
            ),
        }
    except Exception as exc:
        duration = time.perf_counter() - t_start
        retry_count = state.get("retry_count", 0)
        node_label = f"execute_sql_fail_{retry_count}"
        return {
            "result_df": None,
            "sql_error": str(exc),
            "retry_count": retry_count + 1,
            "final_status": "error",
            "node_timestamps": _merge_timestamps(
                state.get("node_timestamps", {}), node_label, iso_start, _now_iso(), duration
            ),
        }


async def format_output(state: AgentState) -> dict:
    """Terminal node — nothing to transform, df is already on state."""
    iso_now = _now_iso()
    return {
        "node_timestamps": _merge_timestamps(
            state.get("node_timestamps", {}), "format_output", iso_now, iso_now, 0.0
        ),
    }


# ──────────────────────────────────────────────────────────────────────
# Conditional edges
# ──────────────────────────────────────────────────────────────────────

def should_retry(state: AgentState) -> str:
    if not state.get("sql_error"):
        return "success"
    if state.get("retry_count", 0) >= MAX_SQL_RETRIES:
        return "max_retries"
    return "retry"


# ──────────────────────────────────────────────────────────────────────
# Graph assembly
# ──────────────────────────────────────────────────────────────────────

def build_graph() -> StateGraph:
    g = StateGraph(AgentState)

    # Nodes
    g.add_node("parallel_search", parallel_search)
    g.add_node("fetch_world_view", fetch_world_view)
    g.add_node("distill_context", distill_context)
    g.add_node("generate_sql", generate_sql)
    g.add_node("execute_sql", execute_sql)
    g.add_node("format_output", format_output)

    # Edges
    g.set_entry_point("parallel_search")
    g.add_edge("parallel_search", "fetch_world_view")
    g.add_edge("fetch_world_view", "distill_context")
    g.add_edge("distill_context", "generate_sql")
    g.add_edge("generate_sql", "execute_sql")

    g.add_conditional_edges(
        "execute_sql",
        should_retry,
        {
            "success": "format_output",
            "retry": "generate_sql",
            "max_retries": "format_output",
        },
    )

    g.add_edge("format_output", END)

    return g.compile()


# ──────────────────────────────────────────────────────────────────────
# Public entry point
# ──────────────────────────────────────────────────────────────────────

graph = build_graph()


async def run_sql_agent(user_query: str) -> dict:
    """
    Run the full text-to-SQL pipeline for a natural language question.

    Returns dict with keys:
      - success: bool
      - query: the user question
      - generated_sql: final SQL
      - result_df: pandas DataFrame (or None)
      - retries: number of SQL retries used
      - error: last error message if failed
      - context: distilled world-view context
      - node_timestamps: per-node timing breakdown
    """
    initial_state: AgentState = {
        "user_query": user_query,
        "keyword_names": [],
        "kpi_names": [],
        "qb_names": [],
        "raw_world_view": {},
        "context": {},
        "generated_sql": "",
        "sql_error": "",
        "retry_count": 0,
        "result_df": None,
        "final_status": "",
        "node_timestamps": {},
    }

    final_state = await graph.ainvoke(initial_state)

    return {
        "success": final_state["final_status"] == "success",
        "query": user_query,
        "generated_sql": final_state["generated_sql"],
        "result_df": final_state["result_df"],
        "retries": final_state.get("retry_count", 0),
        "error": final_state.get("sql_error", ""),
        "context": final_state.get("context", ""),
        "node_timestamps": final_state.get("node_timestamps", {}),
    }

In [ ]:
result = await run_sql_agent("For each Region and General Contractor, calculate the total number of active New Site Build projects, the On-Air Completion Rate (%), the average Site Completion Cycle Time from Construction Start to On-Air, and the count of sites where Civil Start was delayed by more than 60 days after Entitlement Completion.")

print(f"Success: {result['success']}")
print(f"Retries: {result['retries']}")
if result["error"]:
    print(f"Error:   {result['error']}")
print(f"\nGenerated SQL:\n{result['generated_sql']}")

if result["result_df"] is not None:
    print(f"\nResults: {len(result['result_df'])} rows")
    display(result["result_df"])

# ── Node Timestamps ──
print(f"\n{'='*60}")
print("Node Execution Timestamps:")
print(f"{'='*60}")
for node, ts in result["node_timestamps"].items():
    print(f"  {node:<30} | start: {ts['start']} | end: {ts['end']} | duration: {ts['duration_s']:.3f}s")

In [22]:
result

{'success': True,
 'query': 'For each Region and General Contractor, calculate the total number of active New Site Build projects, the On-Air Completion Rate (%), the average Site Completion Cycle Time from Construction Start to On-Air, and the count of sites where Civil Start was delayed by more than 60 days after Entitlement Completion.',
 'generated_sql': 'SELECT\n  rgn_region AS "Region",\n  m_market AS "General Contractor",\n  COUNT(DISTINCT pj_project_id) AS "Total Active New Site Build Projects",\n  ROUND(\n    100.0 * COUNT(DISTINCT pj_project_id) FILTER (WHERE pj_completion_objective_actual_date IS NOT NULL)\n    / NULLIF(COUNT(DISTINCT pj_project_id), 0)\n  , 2) AS "On-Air Completion Rate (%)",\n  ROUND(\n    AVG((pj_completion_objective_actual_date::date - pj_a_4225_construction_start_finish::date))\n    FILTER (WHERE pj_completion_objective_actual_date IS NOT NULL AND pj_a_4225_construction_start_finish IS NOT NULL)\n  , 2) AS "Avg Site Completion Cycle Time (Days)",\n  COU